# Descripción de la notebook

- Levanta el archivo CSV generado en la notebook 0_2.Generando el data set. Elimina las filas que tengan algún campo vacío.
- Luego le aplica transformaciones a los datos:
<ul>
    <li>Transforma los valores enteros del dato 'key' a los valores strings que representan las escalas musicales</li>
    <li>Estandariza el dato string 'album_release_date' a fecha y lo guarda en el campo 'release_date'</li>
    <li>Transforma el dato 'artista_generos' (que contiene un string de géneros separados por coma):
        <ul>
            <li>Se obtienen todos los géneros sin repetir y se los estandariza a ciertos valores para reducir los datos con los que trabajamos</li>
            <li>Por cada uno de esos géneros estandar se crea una columna a modo de dummy</li>
            <li>Por cada registro, se analiza el dato 'artista_generos' y se setean a 1 los dummies correspondientes</li>               </ul>
    </li>
    <li>Dummificamos la columna 'explicit' que es de tipo booleano</li>
    <li>Eliminamos columnas innecesarias</li>
</ul>
- Definimos un subset para testeo solamente (los tracks del año 2020) y lo sacamos del dataframe principal
- Generamos dos nuevos CSV (ambos con las transformaciones): uno para el entrenamiento de los modelos y el otro para casos de testeo (los tracks del 2020)

In [1]:
from sys import maxsize #para imprimir arrays completos
import numpy as np
import pandas as pd
from datetime import datetime

# Levantamos el Dataframe

In [2]:
pathArchivoDataSet = 'tracks_version_definitiva.csv' 
df = pd.read_csv(pathArchivoDataSet)

In [3]:
df.head()

,popularity,id,name,album,duration_ms,album_release_date,explicit,danceability,energy,key,...,acousticness,instrumentalness,liveness,valence,tempo,artista_id,artista_generos,artista_followers,artista_name,time_signature
0,64,6rg1MBZqggsQ5olFGTw0rr,Spaghetti del Rock,"Narigón del Siglo, Yo Te Dejo Perfumado en la ...",212880,2000-02-14,False,0.626,0.389,4,...,0.67700,0.000011,0.117,0.656,97.708,6ZIgPKHzpcswB8zh7sRIhx,"argentine rock,latin alternative,latin rock,ro...",568226,Divididos,4
1,58,6KzeUBu1BE8q24CblG6oku,Tramposa Y Mentirosa,Un Homenaje Al Cielo,212400,2000-05-08,False,0.562,0.756,11,...,0.21700,0.000000,0.150,0.841,87.771,2Mu8h5sFkOziL0Rfn7FXIA,"argentine rock,cumbia santafesina,cumbia villera",515245,Leo Mattioli,4
2,59,0NELwHhQu4EVcmGmu8D7vV,Toco Y Me Voy,Hijos Del Culo,249933,2000-01-01,False,0.641,0.676,5,...,0.32600,0.000000,0.101,0.461,89.991,6MxyNXnnmwQwdW2PD0gXYO,"argentine rock,latin alternative,latin rock,ro...",483671,Bersuit Vergarabat,4
3,58,1g0mhq4ZDQE3cTd1XLcYVf,Sapo de Otro Pozo,Fulanos de Nadie,259146,2000-06-27,False,0.632,0.771,7,...,0.50300,0.000000,0.141,0.723,113.230,6Iv9dXeKX45ff7qe0LDuFW,"argentine rock,latin alternative,latin rock,ro...",194598,Los Caballeros De La Quema,4
4,81,3AJwUDP919kvQ9QcozQPxg,Yellow,Parachutes,266773,2000-07-10,False,0.429,0.661,11,...,0.00239,0.000130,0.234,0.285,173.365,4gzpq5DPGxSnKTe4SA8HAU,"permanent wave,pop",25956508,Coldplay,4


In [4]:
df.shape

(30465, 23)

# Data Cleaning

##### Analizamos los campos vacíos que tiene nuestro dataframe. Podemos ver que está completo, a excepción de la columna 'artista_generos' que tiene algunos valores faltantes

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30465 entries, 0 to 30464
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   popularity          30465 non-null  int64  
 1   id                  30465 non-null  object 
 2   name                30465 non-null  object 
 3   album               30465 non-null  object 
 4   duration_ms         30465 non-null  int64  
 5   album_release_date  30465 non-null  object 
 6   explicit            30465 non-null  bool   
 7   danceability        30465 non-null  float64
 8   energy              30465 non-null  float64
 9   key                 30465 non-null  int64  
 10  loudness            30465 non-null  float64
 11  mode                30465 non-null  int64  
 12  speechiness         30465 non-null  float64
 13  acousticness        30465 non-null  float64
 14  instrumentalness    30465 non-null  float64
 15  liveness            30465 non-null  float64
 16  vale

In [6]:
df_empty_genre = df[df['artista_generos'].isna()]
print('Total de registros que no tienen el dato de los géneros: ' + str(df_empty_genre['artista_name'].count()))

Total de registros que no tienen el dato de los géneros: 882


##### Tenemos 877 registros que no tienen artistas, los eliminamos del data frame principal. Ahora el dataframe nos queda sin valores nulos

In [7]:
df = df[~df['artista_generos'].isna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29583 entries, 0 to 30464
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   popularity          29583 non-null  int64  
 1   id                  29583 non-null  object 
 2   name                29583 non-null  object 
 3   album               29583 non-null  object 
 4   duration_ms         29583 non-null  int64  
 5   album_release_date  29583 non-null  object 
 6   explicit            29583 non-null  bool   
 7   danceability        29583 non-null  float64
 8   energy              29583 non-null  float64
 9   key                 29583 non-null  int64  
 10  loudness            29583 non-null  float64
 11  mode                29583 non-null  int64  
 12  speechiness         29583 non-null  float64
 13  acousticness        29583 non-null  float64
 14  instrumentalness    29583 non-null  float64
 15  liveness            29583 non-null  float64
 16  vale

## Escala musical del track

##### Spotify usa números enteros para definir la escala en la que fue compuesta la canción. La relación entre ese número y las escala es la siguiente : 0 = C, 1 = C♯/D♭, 2 = D, etc (esto es 0 = Do, 1 = Do# o Re bemol, 2 = Re, 3 = Re#, etc). Realizamos una transformación para pasar del valor númerico a un string para que sea mas entendible y pueda luego dummificarse

In [8]:
#agregamos al df una nueva columna 'escala'
df['escala']= ''

In [9]:
#metodo que recibe una fila del dataset y en base al valor de la columna key le setea el valor correspondiente 
#a la columna escala
def translate_key(dataframeRow):
    #Spotify especifica que usa la siguiente notación: 0 = C, 1 = C♯/D♭, 2 = D, etc para las escalas
    #definimos un diccionario para transformar la notación numérica a una notación codificada en strings
    dicc_escalas = { 0 : 'Do', 1: 'Do#', 2: 'Re', 3: 'Re#', 4: 'Mi', 5: 'Fa', 6: 'Fa#', 7: 'Sol', 8: 'Sol#', 9: 'La', 10: 'La#', 11: 'Si', 12: 'Si#', 13: 'Do' }
    dataframeRow.escala = dicc_escalas[dataframeRow.key]
    return dataframeRow

In [10]:
#aplicamos la traducción del valor numérico que representa la escala a un string
df = df.apply(translate_key, axis=1)

In [11]:
df.loc[:, ['key','escala']].head()

,key,escala
0,4,Mi
1,11,Si
2,5,Fa
3,7,Sol
4,11,Si


In [12]:
#sacamos del dataframe la columna original 'key'
df = df.drop(labels='key', axis = 1)

## Año de creación del track

##### Generamos una columna nueva para almacenar la fecha de lanzamiento del track en formato DateTime

In [13]:
df['release_date'] = datetime.today()

In [14]:
#metodo que recibe una fila del data frame y transforma la fecha en formato string de la columna album_release_date
#al formato DateTime y lo guarda en release_date
def cast_string_to_date(dataframeRow):
    try:
        dataframeRow.release_date = datetime.strptime(dataframeRow.album_release_date, '%Y-%m-%d').date()
    except:
        try:
            dataframeRow.release_date = datetime.strptime(dataframeRow.album_release_date, '%Y').date()
        except:
            print("No se pudo convertir string en fecha: " + dataframeRow.album_release_date)
        
    return dataframeRow

In [15]:
df = df.apply(cast_string_to_date, axis=1)

##### Eliminamos la columna fecha en formato string

In [16]:
df = df.drop(labels='album_release_date', axis = 1)

## Géneros

#### Spotify asocia los géneros a los artistas, no a sus tracks. Es por eso que en el dataframe, todas las filas que tengan el mismo artista tendrán los mismos géneros


In [17]:
artist_name = 'Divididos'
artist_divididos = df[df['artista_name']==artist_name]
print('Tenemos ' + str(artist_divididos.shape[0]) + ' registros (tracks) para el artista ' + artist_name)

Tenemos 89 registros (tracks) para el artista Divididos


In [18]:
#Podemos ver que el género está dado por artista, obteniendo el dato artista_generos y viendo que solo tiene un valor único
#para todos los registros (cada registro es un track)
artist_divididos['artista_generos'].unique()

array(['argentine rock,latin alternative,latin rock,rock en espanol,rock nacional,ska argentino'],
      dtype=object)

In [19]:
#Obteniendo los géneros sin repetir, obtenemos 3413
array_of_strings = df['artista_generos'].unique()
print('Buscando los conjuntos de géneros sin repetir tenemos en el data frame, un total de: ' + str(array_of_strings.shape[0]))

Buscando los conjuntos de géneros sin repetir tenemos en el data frame, un total de: 3414


In [20]:
#Ejemplo de un dato género
array_of_strings[0]

'argentine rock,latin alternative,latin rock,rock en espanol,rock nacional,ska argentino'

##### Vamos a tener que separar el conjunto de géneros en géneros individuales (sin repetir)

In [21]:
#generamos un Set para obtener los géneros individuales
genres_set = set()

for concatenated_genres in array_of_strings:
    split_genres = concatenated_genres.split(',')
    
    for genre in split_genres:
        genres_set.add(genre)

print('Vemos que en total son ' + str(len(genres_set)) + ' géneros')

Vemos que en total son 1524 géneros


##### Hacemos una reducción de los géneros, haciendo una transformación de los mismos a unas categorías definidas

In [22]:
#metodo que recibe un string con un género musical y lo reduce a un género musical mas general
def reduce_genre(general_genre):
    genre = ''
    if(genre == '' and 'rock' in general_genre): return 'rock'
    if(genre == '' and 'pop' in general_genre): return 'pop'
    if(genre == '' and 'alternative' in general_genre): return 'alternative'
    if(genre == '' and 'hip hop' in general_genre): return 'hip hop'
    if(genre == '' and 'hiphop' in general_genre): return 'hip hop'
    if(genre == '' and 'hip-hop' in general_genre): return 'hip hop'
    if(genre == '' and 'ska' in general_genre): return 'ska'
    if(genre == '' and 'indie' in general_genre): return 'indie'
    if(genre == '' and 'jazz' in general_genre): return 'jazz'
    if(genre == '' and 'metal' in general_genre): return 'metal'
    if(genre == '' and 'trap' in general_genre): return 'trap'
    if(genre == '' and 'ska' in general_genre): return 'ska'
    if(genre == '' and 'reggaeton' in general_genre): return 'reggaeton'
    if(genre == '' and 'reggae' in general_genre): return 'reggae'
    if(genre == '' and 'blues' in general_genre): return 'blues'
    if(genre == '' and 'rap' in general_genre): return 'rap'
    if(genre == '' and 'disco' in general_genre): return 'disco'
    if(genre == '' and 'folk' in general_genre): return 'folk'
    if(genre == '' and 'punk' in general_genre): return 'punk'
    if(genre == '' and 'soul' in general_genre): return 'soul'
    if(genre == '' and 'r&b' in general_genre): return 'r&b'
    if(genre == '' and 'techno' in general_genre): return 'techno'
    if(genre == '' and 'trance' in general_genre): return 'trance'
    if(genre == '' and 'rumba' in general_genre): return 'rumba'
    if(genre == '' and 'gothic' in general_genre): return 'gothic'
    if(genre == '' and 'cumbia' in general_genre): return 'cumbia'
    if(genre == '' and 'movie' in general_genre): return 'movie'
    if(genre == '' and 'electr' in general_genre): return 'electronic'
    if(genre == '' and 'house' in general_genre): return 'house'
    if(genre == '' and 'christian' in general_genre): return 'christian'
    if(genre == '' and 'under' in general_genre): return 'under'
    if(genre == '' and 'groove' in general_genre): return 'groove'
    if(genre == '' and 'funk' in general_genre): return 'funk'
    if(genre == '' and 'anime' in general_genre): return 'anime'
    if(genre == '' and 'ambient' in general_genre): return 'ambient'
    if(genre == '' and 'country' in general_genre): return 'country'
    if(genre == '' and 'dance' in general_genre): return 'dance'
    if(genre == '' and 'bolero' in general_genre): return 'bolero'
    if(genre == '' and 'bossa nova' in general_genre): return 'bossa_nova'
    if(genre == '' and 'instrumental' in general_genre): return 'instrumental'
    if(genre == '' and 'cassical' in general_genre): return 'classical'
    if(genre == '' and 'melod' in general_genre): return 'melodic'
    if(genre == '' and 'hardcore' in general_genre): return 'hardcore'
    if(genre == '' and 'deep' in general_genre): return 'deep'
    if(genre == '' and 'dubstep' in general_genre): return 'dubstep'
    if(genre == '' and 'dub' in general_genre): return 'dub'
    if(genre == '' and 'experimental' in general_genre): return 'experimental'
    if(genre == '' and 'flamenco' in general_genre): return 'flamenco'
    if(genre == '' and 'freestyle' in general_genre): return 'freestyle'
    if(genre == '' and 'mambo' in general_genre): return 'mambo'
    if(genre == '' and 'merengue' in general_genre): return 'merengue'
    if(genre == '' and 'salsa' in general_genre): return 'salsa'
    if(genre == '' and 'tango' in general_genre): return 'tango'
    if(genre == '' and 'new' in general_genre): return 'neo'
    if(genre == '' and 'neo' in general_genre): return 'neo'
    if(genre == '' and 'nu ' in general_genre): return 'neo'
    if(genre == '' and 'post' in general_genre): return 'neo'
    if(genre == '' and 'opera' in general_genre): return 'opera'
    if(genre == '' and 'rave' in general_genre): return 'rave'
    if(genre == '' and 'progres' in general_genre): return 'progressive'
    if(genre == '' and 'salsa' in general_genre): return 'salsa'
    if(genre == '' and 'samba' in general_genre): return 'samba'
    if(genre == '' and 'swing' in general_genre): return 'swing'
    if(genre == '' and 'trance' in general_genre): return 'trance'
    if(genre == '' and 'world' in general_genre): return 'world'
        
    #si no matcheo con ninguno, mantenemos el genero tal cual es
    if(genre==''):
        genre = 'others'
    
    return genre            

In [23]:
#generamos un nuevo set, aplicándole al set que contiene los géneros generales la transformación para obtener nuestros géneros
#seleccionados como categorias
set_filtrado = set()

for x in genres_set:
    genero_normalizado = reduce_genre(x)
    set_filtrado.add(genero_normalizado)    

##### Con el set de géneros obtenidos los agregamos al data frame como nuevas columnas dummies

In [24]:
for x in set_filtrado:
    df['genre_' + x] = 0

In [25]:
#metodo que recibe una fila del data frame, analiza los generos asociados al artista y setea a 1 los géneros categóricos 
#dummificados
def set_dummy_values(dataframeRow):
    #spliteamos el listado de generos, separados por coma
    row_genres_split = dataframeRow.artista_generos.split(',')
    row_genres_set = set()
    
    #por cada genero aplicamos la normalizacion de géneros para reducirlos a las categorías que definimos
    for x in row_genres_split:
        row_genres_set.add(reduce_genre(x))
    
    #obtenidas las categorías, seteamos a 1 los valores en las columnas dummies
    for x in row_genres_set:
        dataframeRow['genre_'+x] = 1
        
    return dataframeRow

In [26]:
#aplicamos el seteo de los dummies de los géneros
df = df.apply(set_dummy_values, axis=1)

##### Eliminamos la columna que trae todos los géneros concatenados

In [27]:
df = df.drop(labels='artista_generos', axis = 1)

# Dummificamos la columna Explicit

##### Esta columna es booleana, la pasamos a numérica para dummificarla

In [28]:
df['explicit'] = df['explicit'].astype(int)

# Eliminamos otras columnas que no son de utilidad

In [29]:
df = df.drop(labels='id', axis = 1)
df = df.drop(labels='artista_id', axis = 1)

In [30]:
df.columns

Index(['popularity', 'name', 'album', 'duration_ms', 'explicit',
       'danceability', 'energy', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'artista_followers', 'artista_name', 'time_signature', 'escala',
       'release_date', 'genre_ska', 'genre_flamenco', 'genre_metal',
       'genre_opera', 'genre_samba', 'genre_anime', 'genre_groove',
       'genre_indie', 'genre_bossa_nova', 'genre_trance', 'genre_rumba',
       'genre_gothic', 'genre_r&b', 'genre_under', 'genre_christian',
       'genre_folk', 'genre_reggae', 'genre_alternative', 'genre_rock',
       'genre_dubstep', 'genre_mambo', 'genre_dance', 'genre_country',
       'genre_funk', 'genre_reggaeton', 'genre_salsa', 'genre_swing',
       'genre_movie', 'genre_house', 'genre_freestyle', 'genre_instrumental',
       'genre_hip hop', 'genre_world', 'genre_blues', 'genre_neo',
       'genre_merengue', 'genre_disco', 'genre_punk', 'genre_bolero',
       'genr

# Definimos un subset de datos para testeo

### Separamos del set de datos todos los temas que fueron lanzados en el año 2020, para tener un subset de datos exclusivos para el testeo

In [31]:
fecha_inicio_testing =  datetime.strptime('2020-01-01', '%Y-%m-%d').date()
fecha_final_testing =  datetime.strptime('2020-12-31', '%Y-%m-%d').date()

In [32]:
df_test_exclusivo = df[(df['release_date'] >= fecha_inicio_testing) & (df['release_date'] <= fecha_final_testing)]

In [33]:
df_test_exclusivo.shape

(450, 77)

##### Quitamos del dataframe de entrenamiento el subset exclusivo de testing

In [34]:
df = df.drop(axis=0, index=df_test_exclusivo.index)

In [35]:
df.shape

(29133, 77)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29133 entries, 0 to 29999
Data columns (total 77 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   popularity          29133 non-null  int64  
 1   name                29133 non-null  object 
 2   album               29133 non-null  object 
 3   duration_ms         29133 non-null  int64  
 4   explicit            29133 non-null  int32  
 5   danceability        29133 non-null  float64
 6   energy              29133 non-null  float64
 7   loudness            29133 non-null  float64
 8   mode                29133 non-null  int64  
 9   speechiness         29133 non-null  float64
 10  acousticness        29133 non-null  float64
 11  instrumentalness    29133 non-null  float64
 12  liveness            29133 non-null  float64
 13  valence             29133 non-null  float64
 14  tempo               29133 non-null  float64
 15  artista_followers   29133 non-null  int64  
 16  arti

# Generamos el CSV con los datos limpios

In [37]:
tracks_csv_name = 'tracks_clean.csv'
df.to_csv(tracks_csv_name,index=False,encoding='utf-8')

# Generamos un CSV con los datos de testeo

In [38]:
tracks_csv_name = 'tracks_clean_test.csv'
df_test_exclusivo.to_csv(tracks_csv_name,index=False,encoding='utf-8')